In [1]:
import os
from bottle import route, run, get, error
from pymongo import MongoClient
from dotenv import load_dotenv
import dns
import json
import pymongo
from bson.json_util import dumps, loads
from textblob import TextBlob

load_dotenv()
url=os.getenv('password')
client = MongoClient(url)
db = client['chat']
coll = db['chat-prueba']

In [2]:
def sentiments():
    sentiment=dict()
    text=''
    for e in coll.find():
        text+=e['text']+' '
    sentiment['analysis']=TextBlob(text).sentiment
    return sentiment

In [3]:
sentiments()

{'analysis': Sentiment(polarity=0.051466836734693884, subjectivity=0.5876275510204082)}

In [4]:
text=''
for e in coll.find():
    text+=e['text']+' '
print(text)

TextBlob('\\ud83d').sentiment

Hey Mike, whats up?? Dude!!! 😀️ Did you watch the game last night? No, had to work. How was it? Awesome! Boyander scored 3 goals! No way! 😮️ Way... I bet people went crazy For sure. We stayed at KuboLoco until 6AM. Motherfucker! Hungover? 😵️ Dead, burried and still suffering! 🤮️ Im so angry right now! I saw you this morning. You shouldnt get so upset. ❤️ Impossible! Cant do it! Just cant! He pissed me off so much. Dont get so frustrated, Leia. Remember what German said. But how am I suposed to fix the facial recognition system if it doesnt even work? Im sure you will find a way. And who the hell is Tony? 🤯️ I am Iron Man Oh shit. I think youre friends with my brother. Uh-Oh. Better fix that quickly. Sweety pie! How Ive longed to talk to you. Not again, Michael.... 😒️ Its Mike, honey bunny. Or doctor LOVE! And youre a sick little girl. Im gonna report you to Human Resources again, Michael. Cant handle me on your own? Id rather have a fucking alien crawling out of my ass That would be ho

Sentiment(polarity=0.0, subjectivity=0.0)

In [5]:
def sentiment_of_everyone():
    conversation=dict()
    conv_analysis=dict()
    for e in coll.find():
        if e['userName'] not in conversation:
            conversation[e['userName']]=e['text']
        else:
            conversation[e['userName']]+=' '+ e['text']
    for e in conversation:
        conv_analysis[e]=TextBlob(conversation[e]).sentiment
    return conv_analysis

In [6]:
sentiment_of_everyone()

{'John Wick': Sentiment(polarity=0.15625, subjectivity=0.5972222222222222),
 'Mike Wazowski': Sentiment(polarity=0.13607142857142857, subjectivity=0.6562698412698413),
 'Leia Organa': Sentiment(polarity=-0.19047619047619047, subjectivity=0.5892857142857143),
 'Tony Stark': Sentiment(polarity=0.12666666666666665, subjectivity=0.4177777777777777),
 'Ellen Ripley': Sentiment(polarity=0.125, subjectivity=0.45),
 'J. Jonah Jameson': Sentiment(polarity=-0.6402777777777778, subjectivity=0.7222222222222222),
 'Danny Torrances Finger': Sentiment(polarity=0.2875, subjectivity=0.7000000000000001),
 'Danny Torrance': Sentiment(polarity=0.02321428571428571, subjectivity=0.5410714285714285)}

In [7]:
def sentiment_user(user):
    conversation=dict()
    conv_analysis=dict()
    for e in coll.find():
        if e['userName']==user:
            conversation[user]=e['text']
        else:
            conversation[user]+=' '+e['text']
    conv_analysis[user]=TextBlob(conversation[user]).sentiment
    return conv_analysis

In [8]:
sentiment_user('John Wick')

{'John Wick': Sentiment(polarity=0.17604166666666665, subjectivity=0.5906250000000001)}

In [9]:
"""conversation=dict()

for e in coll.find():
    if e['userName'] not in conversation:
        conversation[e['userName']]=e['text']
    else:
        conversation[e['userName']]+=e['text']
conversation
dumps(conversation)"""

"""conversation_analysis=dict()

for e in conversation:
    conversation_analysis[e]=TextBlob(conversation[e]).sentiment
conversation_analysis"""

'conversation_analysis=dict()\n\nfor e in conversation:\n    conversation_analysis[e]=TextBlob(conversation[e]).sentiment\nconversation_analysis'

In [10]:
def getting_x_messages(y):
    y=int(y)
    return dumps(coll.find({},{'userName':1,'text':1}).limit(y))


In [11]:
(getting_x_messages(2))

'[{"_id": {"$oid": "5de259b414b34255d4a5c7bc"}, "userName": "John Wick", "text": "Hey Mike, whats up??"}, {"_id": {"$oid": "5de259b414b34255d4a5c7bd"}, "userName": "Mike Wazowski", "text": "Dude!!! \\ud83d\\ude00\\ufe0f Did you watch the game last night?"}]'

In [12]:
import nltk # install nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

sid.polarity_scores("Weekend proyect")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/guzman/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [13]:
sid.polarity_scores('I am sad')

{'neg': 0.756, 'neu': 0.244, 'pos': 0.0, 'compound': -0.4767}

In [14]:
text='I am sad'


In [15]:
from nltk.corpus import wordnet

synonyms = []
antonyms = []

for syn in wordnet.synsets("active"):
    for l in syn.lemmas():
        synonyms.append(l.name())
print(set(synonyms))


{'combat-ready', 'participating', 'alive', 'active_voice', 'fighting', 'active', 'dynamic', 'active_agent'}


In [36]:
import operator
import regex
def improving_sentence(text):
    sentence_lst=text.split()
    sentence_dict=dict()
    polarity_dict=dict()
    polar_scores=[]
    word_to_change=''
    scores_dict=dict()
    for i in range(len(sentence_lst)):
        sentence_dict[i]=sentence_lst[i]
    for i in range(len(sentence_lst)):
        polarity_dict[i]=sid.polarity_scores(sentence_dict[i])
    for i in range(len(sentence_lst)):
        scores_dict[sentence_lst[i]]=polarity_dict[i]['compound']
   
    lst_x=sorted(scores_dict.items(), key=operator.itemgetter(1))
    
    final_dict=dict()
    
    word_to_change=lst_x[0][0]
    
    for e in lst_x:
        final_dict[e[0]]=e[1]
        
    synonyms = []
    
    word=list(final_dict)[0]
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
    
    synonyms_dict=dict()
    
    lst_of_synonyms_scores=[]
    
    for e in synonyms:
        synonyms_dict[e]=sid.polarity_scores(e)['compound']
    lst_of_synonyms=sorted(synonyms_dict.items(),key=operator.itemgetter(1), reverse=True)
    
    if word_to_change=='I':
        text=text
    else:
        text=regex.sub(word_to_change,lst_of_synonyms[0][0],text)
    
    return text

In [40]:
improving_sentence(x[6]['text'])

'I bet people went loony'

In [38]:
x=list(coll.find())

for i in range(len(x)):
    try:
        j=improving_sentence(x[i]['text'])
        print(j)
        
    except:
        pass
    
x[4]['text']

nobelium way! 😮️
I bet people went loony
Im so tempestuous right now!
Impossible! Cant do it! Just cant! He make me off so much.
merely how am I suposed to fix the facial recognition system if it doesnt even work?
And who the hell_on_earth is Tony? 🤯️
I am Iron Man
not again, Michael.... 😒️
Its Mike, honey bunny. Or doctor LOVE! And youre a vomit little girl.
buzzword handle me on your own?
Id rather have a fucking alien crawling out of my buttocks
Ok, youve got to halt drinking! Im calling HR.
I NEED VODKA LIKE A CAR NEEDS GAS
I want pictures of Spiderman and I want them now!
I do! I just dont feel appreciated in my job
I dunno...
possibly
make you mean red room? Id love a red room.
thrill out, pinky.. Its such a nice day
ailment go for a walk in the park, I guess. Maybe fishing
fishing sounds good. What about the labyrinth?


'No way! 😮️'

In [ ]:
def sentiments(x):
    x=int(x)
    sentiment=dict()
    text=''
    z=(coll.find({"idChat":x}))
    for e in z:
        text+=e['text']
    sentiment['analysis']=TextBlob(text).sentiment
    return sentiment

In [ ]:
sentiments(4)